# Obliczenia Naukowe
### Labolatoria, Lista 1<br/>Kamil Matejuk, 250135

## Zad 1

### 1.1
*Napisać program w języku Julia wyznaczający iteracyjnie epsilony maszynowe dla wszystkich dostępnych typów zmiennopozycyjnych Float16, Float32, Float64, zgodnych ze standardem IEEE 754 (half, single, double), i porównać z wartościami zwracanymi przez
funkcje: eps(Float16), eps(Float32), eps(Float64) oraz z danymi zawartymi w pliku
nagłówkowym float.h dowolnej instalacji języka C.*

In [14]:
# Kamil Matejuk, 250135
using Printf

function calculate_eps(dtype)
    macheps = convert(dtype, 1)
    while convert(dtype, 1.0 + macheps) > 1.0 && convert(dtype, 1.0 + macheps) == 1 + macheps
        macheps = convert(dtype, macheps/2)
    end
    return 2 * macheps
end

function compare_eps()
    # dane zawarte w pliku nagłówkowym float.h instalacji języka C
    # https://www.tutorialspoint.com/c_standard_library/float_h.htm
    c_lang_data = (
        Float16 => 1.0e-5,
        Float32 => 1.0e-9,
        Float64 => 1.0e-9)
    println("Data Type              Calculated            Build in eps()                Build in C")
    for (dtype, c_val) in c_lang_data
        println("$dtype $(@sprintf("%25s", calculate_eps(dtype))) $(@sprintf("%25s", eps(dtype))) $(@sprintf("%25s", c_val))")
    end
end
compare_eps()

Data Type              Calculated            Build in eps()                Build in C
Float16                  0.000977                  0.000977                    1.0e-5
Float32              1.1920929e-7              1.1920929e-7                    1.0e-9
Float64     2.220446049250313e-16     2.220446049250313e-16                    1.0e-9


W powyższym programie funkcja `calculate_eps()` iteracyjnie zmiejsza zmienną `macheps` dwukrotnie, aby znaleźć najbliższą odległość od 1 do następnej liczby reprezentowanej w zadanym typie danych, tj spełniającą równania:

    fl(1.0 + macheps) > 1.0
    fl(1.0 + macheps) = 1 + macheps

Funkcja `compare_eps()` służy tylko do porównania, wykorzystując wbudowaną funkcję `eps()` oraz dane z pliku `float.h`, pobrane ze strony https://www.tutorialspoint.com/c_standard_library/float_h.htm.<br/>
Da się zauważyć, że obliczone wyniki są w zaokrąglęniu takie same jak zwracane przez `eps()`. Dane wbudowane w plik `float.h` w języku C są natomiast znacznie mniejsze dla typow `Float16` (ok 10 razy) i `Float32` (ok 100 razy). Dla typu danych `Float64` wartości obliczone są jednak znacznie dokładniejsze (10^7 razy).

### 1.2
*Napisać program w języku Julia wyznaczający iteracyjnie liczbę maszynową eta taką, że eta > 0.0 dla wszystkich dostępnych typów zmiennopozycyjnych Float16, Float32, Float64, zgodnych ze standardem IEEE 754 (half, single, double), i porównać z wartościami zwracanymi przez
funkcje: nextfloat(Float16(0.0)), nextfloat(Float32(0.0)), nextfloat(Float64(0.0)).*

In [15]:
# Kamil Matejuk, 250135
using Printf

function calculate_eta(dtype)
    eta = convert(dtype, 1)
    while convert(dtype, eta/2) > 0.0
        eta = convert(dtype, eta/2)
    end
    return eta
end

function compare_eta()
    types = [Float16, Float32, Float64]
    println("Data Type              Calculated      Build in nextfloat()")
    for dtype in types
        println("$dtype $(@sprintf("%25s", calculate_eta(dtype))) $(@sprintf("%25s", nextfloat(dtype(0.0))))")
    end
end
compare_eta()

Data Type              Calculated      Build in nextfloat()
Float16                    6.0e-8                    6.0e-8
Float32                   1.0e-45                   1.0e-45
Float64                  5.0e-324                  5.0e-324


W powyższym programie funkcja `calculate_eta()` iteracyjnie zmiejsza zmienną `eta` dwukrotnie, aby znaleźć najmniejszą liczbę dodatnią, tj spełniającą równanie:

    fl(eta) > 0.0

Funkcja `compare_eta()` służy tylko do porównania, wykorzystując wbudowaną funkcję `nextfloat()`.
Da się zauważyć, że obliczone wyniki są takie same jak podane przez wbudowaną funkcję `nextfloat()`.

### 1.3
*Jaki związek ma liczba macheps z precyzją arytmetyki (oznaczaną na wykładzie przez $\epsilon$)?*

Precyzja arytmetyka wyznaczana jest jako $\epsilon = 2^{-t-1}$, gdzie $t$ jest liczbą bitów przeznaczoną na mantysę. Precyzja arytmetyki zależy wyłącznie od liczby bitów przeznaczonych na reprezentację mantysy. Zatem dla typów zmiennoprzecinkowych powstanie zależność:

| Typ         | t   | $\epsilon$             | macheps               |
| :---------: | :-: | ---------------------: | --------------------: |
| Float16     | 10  | 0.00048828125          | 0.000977              |
| Float32     | 23  | 5.960464477539063e-8   | 1.1920929e-7          |
| Float64     | 52  | 1.1102230246251565e-16 | 2.220446049250313e-16 |

Porównując z warościami $macheps$ obliczonymi w zad **1.1**, da się zauważyć że precyzja jest 2 razy mniejsza niż epsilon maszyonwy. Dzieje się tak, ponieważ $macheps$ jest najmniejszą odległością pomiędzy dwiema liczbami zapisanymi w danym typie, natomiast precyzja mówi o zakresie jaki jest zaokraglany, a więc połowę odległości pomiędzy liczbami.

### 1.4
*Jaki związek ma liczba eta z liczbą MINsub (zob. wykład 1)?*

Liczba $MIN_{sub}$ wyznaczana jest z wzoru $MIN_{sub} = 2^{−t} \cdot 2^{c_{min}}$, gdzie $t$ jest liczbą bitów przeznaczoną na mantysę, natomiast $c_{min}$ minimalną możliwą wartością cechy. $c_{min} = 2−2^{d−1}$, $d$ - liczba bitów przeznaczona na cechę.

| Typ         | t   | d   | $c_{min}$ | $MIN_{sub}$           | eta      |
| :---------: | :-: | :-: | :-------: |---------------------: | -------: |
| Float16     | 10  | 5   | -14       | 5.960464477539063e-8  | 6.0e-8   |
| Float32     | 23  | 8   | -126      | 1.401298464324817e-45 | 1.0e-45  |
| Float64     | 52  | 11  | -1022     | 5.0e-324              | 5.0e-324 |

Porównując z wartościami $eta$ otrzymanymi w zad **1.2**, da się zauważyć że sa one podobne, co do zaokrąglenia. Dzieje się tak, ponieważ $MIN_{sub}$ zwraca najmniejszą dodatnią liczbę, którą da się przedstawić i która nie jest nieskończonością, co jest dokładnie wartością eta, zgodnie z wzorem z zad 1.2

### 1.5
*Co zwracają funkcje floatmin(Float32) i floatmin(Float64) i jaki jest związek zwracanych wartości z liczbą MINnor (zob. wykład 1)?*

Liczba $MIN_{nor}$ jest większa od $MIN_{sub}$ i wyznaczana jest wzorem $MIN_{nor} = 2^{c_{min}}$.

| Typ         | t   | d   | $c_{min}$ | $MIN_{nor}$             | floatmin()              |
| :---------: | :-: | :-: | :-------: |-----------------------: | ----------------------: |
| Float16     | 10  | 5   | -14       | 6.103515625e-5          | 6.104e-5                |
| Float32     | 23  | 8   | -126      | 1.1754943508222875e-38  | 1.1754944f-38           |
| Float64     | 52  | 11  | -1022     | 2.2250738585072014e-308 | 2.2250738585072014e-308 |

Porównując z wartościami otrzymanymi poprzez `floatmin()`, da się zauważyć że sa one podobne, co do zaokrąglenia. Dzieje się tak, ponieważ $MIN_{nor}$ zwraca najmniejszą dodatnią liczbę, którą da się przedstawić, oraz której cecha jest większa niż $0$ (tj nie występuje niedomiar - *"underflow"*)

### 1.6
*Napisać program w języku Julia wyznaczający iteracyjnie liczbę (MAX) dla wszystkich typów zmiennopozycyjnych Float16, Float32, Float64, zgodnych ze standardem IEEE 754 (half, single, double), i porównać z wartościami zwracanymi przez funkcje: floatmax(Float16), floatmax(Float32), floatmax(Float64) oraz z danymi zawartymi w
pliku nagłówkowym float.h dowolnej instalacji języka C*

In [16]:
# Kamil Matejuk, 250135
using Printf

function calculate_MAX(dtype)
    max = convert(dtype, 1)
    while !isinf(2 * max)
        max = convert(dtype, 2 * max)
    end
    max2 = convert(dtype, max / 2)
    while !isinf(max + max2)
        max = max + max2
        max2 = convert(dtype, max2 / 2)
    end
    return max
end

function compare_MAX()
    # dane zawarte w pliku nagłówkowym float.h instalacji języka C
    # https://www.tutorialspoint.com/c_standard_library/float_h.htm
    c_lang_data = (
        Float16 => 1.0e37,
        Float32 => 1.0e37,
        Float64 => 1.0e37)
    println("Data Type              Calculated       Build in floatmax()                Build in C")
    for (dtype, c_val) in c_lang_data
        println("$dtype $(@sprintf("%25s", calculate_MAX(dtype))) $(@sprintf("%25s", floatmax(dtype))) $(@sprintf("%25s", c_val))")
    end
end
compare_MAX()

Data Type              Calculated       Build in floatmax()                Build in C
Float16                    6.55e4                    6.55e4                    1.0e37
Float32              3.4028235e38              3.4028235e38                    1.0e37
Float64    1.7976931348623157e308    1.7976931348623157e308                    1.0e37


W powyższym programie funkcja `calculate_MAX()` składa się z dwóch etapów. W pierwszym iteracyjnie zwiększa zmienną `max` dwukrotnie, aż do osiągnięcia połowy `inf`. Następnie dodaje stopniowo coraz miejsze części otrzymanej liczby max, tworząc szereg, który dąży do nieskończoności:

*max* + $\frac{1}{2}$ *max* + $\frac{1}{4}$ *max* + $\frac{1}{8}$ *max* + ...

W ten sposób zostanie otrzymana liczba możliwie najbliższa nieskończoności.<br/>
Funkcja `compare_MAX()` służy tylko do porównania, wykorzystując wbudowaną funkcję `floatmax()` oraz dane z pliku `float.h`, pobrane ze strony https://www.tutorialspoint.com/c_standard_library/float_h.htm.<br/>
Da się zauważyć, że obliczone wyniki są takie same jak zwracane przez `floatmax()`. Dane wbudowane w plik `float.h` w języku C są natomiast stałe niezaleznie od typu i około 3 razy większe niż obliczone dla `Float32`.

## Zad 2

*Kahan stwierdził, że epsilon maszynowy (macheps) można otrzymać obliczając wyrażenie 3(4/3−1)−1 w arytmetyce zmiennopozycyjnej. Sprawdzić eksperymentalnie w języku Julia słuszność tego stwierdzenia dla wszystkich typów zmiennopozycyjnych Float16, Float32, Float64.*

In [17]:
# Kamil Matejuk, 250135
using Printf

types = [Float16, Float32, Float64]
println("Data Type              Calculated            Build in eps()                Difference")
for dtype in types
    eps1 = abs(convert(dtype, 3)*(convert(dtype, 4)/convert(dtype, 3)-convert(dtype, 1))-convert(dtype, 1))
    eps2 = eps(dtype)
    delta = eps1 - eps2
    println("$dtype $(@sprintf("%25s", eps1)) $(@sprintf("%25s", eps2)) $(@sprintf("%25s", delta))")
end

Data Type              Calculated            Build in eps()                Difference
Float16                  0.000977                  0.000977                       0.0
Float32              1.1920929e-7              1.1920929e-7                       0.0
Float64     2.220446049250313e-16     2.220446049250313e-16                       0.0


Biorąc pod uwagę tylko wartość bezwzględną wyrażenia 3 $\cdot$ ($\frac{4}{3}$ - 1) - 1, gdzie każda liczba została przekonwertowana na zadany typ danych, wyniki są takie same jak wynik wbudowanej funkcji `eps()`.<br/>
Wynika to z niemożności zapisania dokładnej wartości $\frac{1}{3}$ w systemie binarnym.

## Zad 3

*Sprawdź eksperymentalnie w języku Julia, że w arytmetyce Float64 (arytmetyce double w standarcie IEEE 754) liczby zmiennopozycyjne są równomiernie rozmieszczone w [1, 2] z krokiem δ = $2^{−52}$. Innymi słowy, każda liczba zmiennopozycyjna x pomiędzy 1 i 2 może być przedstawione następująco x = 1 + kδ w tej arytmetyce, gdzie k = 1,2, ... , $2^{52}$ − 1 i δ = $2^{−52}$<br/>
Jak rozmieszczone są liczby zmiennopozycyjne w przedziale [$\frac{1}{2}$, 1], jak w przedziale [2, 4] i jak mogą być przedstawione dla rozpatrywanego przedziału?*

In [20]:
# Kamil Matejuk, 250135
function linear_check_arrangement(from, to)
    x = convert(Float32, from)
    global_delta = abs(x - nextfloat(x))
    i = 0
    while x < to
        delta = abs(x - nextfloat(x))
        if delta != global_delta
            println("Values are not equally distributed")
            return
        end
        x = nextfloat(x)
        i = i + 1
    end
    println("Values are equally distributed, and can be shown as x = $from + k*d, k in [1 ... $i], d = $global_delta")
end
# @time linear_check_arrangement(1, 2)

linear_check_arrangement (generic function with 1 method)

Funkcja `linear_check_arrangement()` jest podejściem linearnym do problemu, tj iteracyjnym przejściem przez każdą liczbę z zakresu oraz sprawdzenie czy odstęp jest niezmienny. Jest to proste rozwiązanie eksperymentalne problemu, oraz dokładnie sprawdzające i działające dla każdego zadanego zakresu. To rozwiązanie można zastosować dla każdego możliwego zakresu i wykaże każdą nieprawidłowość. Problemem tego sposobu jest natomiast złożoność czasowa $O(2^m)$ gdzie $m$ jest długością mantysy. Dla `Float16` oraz `Float32` daje to znośne czasy ($0.003770s$ oraz $0.041107s$ odpowiednio), natomiast przy typie `Float64` przewidywany czas jest już zbyt duży by jakiekolwiek obliczenia miały sens. Dlatego poniżej proponuję lepsze rozwiązanie:

In [21]:
# Kamil Matejuk, 250135
using Random

function split_structure(n)
    n = convert(Float64, n)
    sign = parse(Int, bitstring(n)[1], base=2)
    exp = parse(Int, bitstring(n)[2:12], base=2) - 1023
    mantis = parse(Int, bitstring(n)[13:64], base=2)
    return (sign, exp, mantis)
end

function check_arrangement(from, to)
    from_s, from_e, from_m = split_structure(from)
    to_s, to_e, to_m = split_structure(to)
    if from_s != to_s
        println("Numbers arent equally distributed")
        return
    end
    if from_m == to_m == 0 && abs(from_e - to_e) == 1
        # equal distribution
        i = 2^52
        delta = convert(Float64, abs(from - to) / i)
        for j in 1:10
            r = convert(Float64, rand()*abs(from - to) + from)
            if convert(Float64, abs(r - nextfloat(r))) != delta
                println(r)
                println(nextfloat(r))
                println(abs(r - nextfloat(r)))
                println(delta)
                println("Numbers arent equally distributed")
                return
            end
        end
        println("Numbers are equally distributed, and can be shown as x = $from + k*d, k in [1 ... $i], d = $delta")
    else
        println("Numbers arent equally distributed")
    end
end

check_arrangement(1, 2)
check_arrangement(0.5, 1)
check_arrangement(2, 4)

Numbers are equally distributed, and can be shown as x = 1 + k*d, k in [1 ... 4503599627370496], d = 2.220446049250313e-16
Numbers are equally distributed, and can be shown as x = 0.5 + k*d, k in [1 ... 4503599627370496], d = 1.1102230246251565e-16
Numbers are equally distributed, and can be shown as x = 2 + k*d, k in [1 ... 4503599627370496], d = 4.440892098500626e-16


Podejście powyższe korzysta z wiedzy jak skonstruowane są liczby Float. Każda liczba `Float64` jest zapisywana jako:<br/>
$n = (-1)^s \cdot (1 + m) \cdot 2^e$<br/>
Jeżeli zakresy przedziału mają różne znaki `s` od razu wiadomo, że nie sa równomiernie rozłożone, ponieważ w przedziale zawiera się 0, co wymusza występowanie zera maszynowego. Przypadek takich samych znaków można sprowadzić do przypadku obu dodatnich.<br/>
Jeżeli mantysa `m` jest równa 0 dla początku i końcu przedziału, oraz eksponenty `e` różnią się o 1, oznacza to że każda liczba wewntrz przedziału jest postaci $n = (1 + m) \cdot 2^e$, co oznacza że różnią się tylko wartością na mantysie i są równomiernie rozłożone. Różnych liczb na mantysie jest $2^{52}$ (dla Float64), zatem aby uzyskać rozkład liczb wystarczy podzielić wielkość przedziału na ilość liczb. Nastepnie algorytm sprawdza poprawność, poprzez wykonanie 10 losowych testów: wybór liczby i jej następnika, oraz sprawdzenie czy odległość się zgadza z wyznaczoną. Dla zadanych przykładów:

| Zakres   | from                  | to                  | Postać                                               |
| :------: | :-------------------: | :-----------------: | :--------------------------------------------------: |
| (1,2)    | (1+0)$\cdot$$2^0$     | (1+0)$\cdot$$2^1$   | {1 + k$\cdot$.220446049250313e-16, k $\in$ {0 $\dots$$2^{52}-1$}} |
| (0.5, 1) | (1+0)$\cdot$$2^{-1}$  | (1+0)$\cdot$$2^0$   | {1 + k$\cdot$1.1102230246251565e-16, k $\in$ {0 $\dots$$2^{52}-1$}} |
| (2, 4)   | (1+0)$\cdot$$2^1$     | (1+0)$\cdot$$2^2$   | {1 + k$\cdot$4.440892098500626e-16, k $\in$ {0 $\dots$$2^{52}-1$}} |

Da się zauważyć, że operując w zakresach będących kolejnymi potegami dwójki ($\dots, \frac{1}{4}, \frac{1}{2}, 1, 2, 4, \dots$) liczby są równomiernie rozłożone, z deltą zależną od całkowitego rozmiaru przedziału. W każdym przedziale mieści się taka sama ilość liczb, są one jedynie bardziej lub mniej zagęszczone.


## Zad 4

### 4.1
*Znajdź eksperymentalnie w arytmetyce Float64 zgodnej ze standardem IEEE 754 (double) liczbę zmiennopozycyjną x w przedziale [1, 2], taką, że $x \cdot \frac{1}{x} \neq 1$*

In [22]:
# Kamil Matejuk, 250135

function find_anomaly(from, to)
    from = convert(Float64, from)
    to = convert(Float64, to)
    while from <= to && from*(1/from) == 1
        from = nextfloat(from)
    end
    if from == to
        println("Such number doesn't exists between $from and $to")
    else
        result = from*(1/from)
        diff = abs(from - result)
        println("Number $from results in $result after given condition, with difference in value of $diff")
    end
end

find_anomaly(1, 2)

Number 1.000000057228997 results in 0.9999999999999999 after given condition, with difference in value of 5.7228997207836585e-8


Przeszukując od najmniejszej liczby pojedyńczo (korzystając z iteratora `nextfloat`) aż do spełnienia warunku anomalii. Wynikiem i najmniejszą liczbą spełniającą warunek jest 1.000000057228997, ponieważ:<br/>
$1..000000057228997 \cdot \frac{1}{1.000000057228997} = 0.9999999999999999 \neq 1$

### 4.2
*Znajdź najmniejszą taką liczbę.*

In [23]:
# Kamil Matejuk, 250135

function find_min_anomaly()
    i = convert(Float64, 0)
    i = nextfloat(i)
    while i*(1/i) == 1
        i = nextfloat(i)
    end
    result = i*(1/i)
    diff = abs(i - result)
    println("Number $i results in $result after given condition")
end

find_min_anomaly()

Number 5.0e-324 results in Inf after given condition


Przeszukując od zera (korzystając z iteratora `nextfloat`) aż do spełnienia warunku anomalii. Wynikiem i najmniejszą liczbą spełniającą warunek jest 5.0e-324, ponieważ:<br/>
$(5.0e-324) \cdot \frac{1}{5.0e-324} = Inf \neq 1$

## Zad 5

*Napisz program w języku Julia realizujący następujący eksperyment obliczania iloczynu skalarnego dwóch wektorów:<br/>
x = [2.718281828, −3.141592654, 1.414213562, 0.5772156649, 0.3010299957]<br/>
y = [1486.2497, 878366.9879, −22.37492, 4773714.647, 0.000185049]<br/>
Zaimplementuj cztery algorytmy i policz sumę na cztery sposoby dla n = 5 (“w przód”, “w tył”, od największego do najmniejszego, od najmniejszego do największego). Użyj pojedynczej i podwójnej precyzji (typy Float32 i Float64 w języku Julia). Porównaj wyniki z prawidłową wartością (dokładność do 15 cyfr) $−1.00657107000000 \cdot 10^{−11}$*

In [24]:
# Kamil Matejuk, 250135
using Printf

function scalar_product_1(dtype, arr1, arr2)
    @assert length(arr1) == length(arr2)
    sum = convert(dtype, 0)
    for i in 1:length(arr1)
        i1 = convert(dtype, arr1[i])
        i2 = convert(dtype, arr2[i])
        sum = sum + i1*i2
    end
    return sum
end

function scalar_product_2(dtype, arr1, arr2)
    @assert length(arr1) == length(arr2)
    sum = convert(dtype, 0)
    for i in 1:length(arr1)
        j = length(arr1) - i + 1
        i1 = convert(dtype, arr1[j])
        i2 = convert(dtype, arr2[j])
        sum = sum + i1*i2
    end
    return sum
end

function scalar_product_3(dtype, arr1, arr2)
    @assert length(arr1) == length(arr2)
    arr = []
    for i in 1:length(arr1)
        i1 = convert(dtype, arr1[i])
        i2 = convert(dtype, arr2[i])
        push!(arr, i1*i2)
    end
    arr = sort(arr) # smallest to highest
    sum = convert(dtype, 0)
    for i in 1:length(arr)
        j = length(arr) - i + 1
        sum = sum + arr[j]
    end
    return sum
end

function scalar_product_4(dtype, arr1, arr2)
    @assert length(arr1) == length(arr2)
    arr = []
    for i in 1:length(arr1)
        i1 = convert(dtype, arr1[i])
        i2 = convert(dtype, arr2[i])
        push!(arr, i1*i2)
    end
    arr = sort(arr) # smallest to highest
    sum = convert(dtype, 0)
    for i in arr
        sum = sum + i
    end
    return sum
end

function compare_scalars()
    x = [2.718281828, -3.141592654, 1.414213562, 0.5772156649, 0.3010299957]
    y = [1486.2497, 878366.9879, -22.37492, 4773714.647, 0.000185049]
    correct = -1.00657107000000e-11
    println("Correct Result: $correct")
    println("Data Type             Function          Calucated Result                Difference")
    for dtype in [Float32, Float64]
        for f in [scalar_product_1, scalar_product_2, scalar_product_3, scalar_product_4]
            s = f(dtype, x, y)
            diff = abs(s - correct)
            println("$dtype       $f $(@sprintf("%25s", s)) $(@sprintf("%25s", diff))")
        end
    end
end
compare_scalars()

Correct Result: -1.00657107e-11
Data Type             Function          Calucated Result                Difference
Float32       scalar_product_1                -0.4999443       0.49994429944939167
Float32       scalar_product_2                -0.4543457        0.4543457031149343
Float32       scalar_product_3                      -0.5        0.4999999999899343
Float32       scalar_product_4                      -0.5        0.4999999999899343
Float64       scalar_product_1    1.0251881368296672e-10    1.1258452438296672e-10
Float64       scalar_product_2   -1.5643308870494366e-10    1.4636737800494365e-10
Float64       scalar_product_3                       0.0            1.00657107e-11
Float64       scalar_product_4                       0.0            1.00657107e-11


Użycie funkcji 3 oraz 4, tj "od największego do najmniejszego" oraz "od najmniejszego do największego" spowodowało duże zaokrąglenie wyników, do odpowiednio $0.5$ dla `Float32` oraz $0.0$ dla `Float64`. Dodawanie w porządku losowym pozwoliło na nieprzeciążanie sumy na jedną stronę, np poprzez dodawanie samych dużych liczb. Wnioskiem jest, że największy błąd kumuluje się na początku, dlatego najbardziej optymalnie jest na początku wykonywać działania na mniejszych liczbach, potem na większych, aby zminimalizować łączne błędy arytmetyki. Zauważyć można natomaist, że wyniki się różnią, zatem kolejność sumowania jest ważna.

## Zad 6

*Policz w języku Julia w arytmetyce Float64 wartości następujących funkcji<br/>
$f(x) = \sqrt{x^2 + 1} - 1$<br/>
$g(x) = x^2 / (\sqrt{x^2 + 1} + 1)$<br/>
dla kolejnych wartości argumentu $x = 8^{−1}, 8^{−2}, 8^{−3} ...$ Chociaż f = g komputer daje różne wyniki. Które z nich są wiarygodne, a które nie?*

In [25]:
# Kamil Matejuk, 250135
using Printf

function f(x)
    x = convert(Float64, x)
    return (x^2 + 1)^0.5 - 1
end

function g(x)
    x = convert(Float64, x)
    return x^2 / ((x^2 + 1)^0.5 + 1)
end

println("x                                     f(x)                      g(x)                   difference")
for i in 1:20
    x = 1 / 8^i
    fx = f(x)
    gx = g(x)
    diff = abs(fx - gx)
    println("$(@sprintf("%-25s", x)) $(@sprintf("%25s", fx)) $(@sprintf("%25s", gx)) $(@sprintf("%25s", diff))")
end

x                                     f(x)                      g(x)                   difference
0.125                         0.0077822185373186414     0.0077822185373187065     6.505213034913027e-17
0.015625                     0.00012206286282867573    0.00012206286282875901     8.328027937404281e-17
0.001953125                   1.9073468138230965e-6      1.907346813826566e-6     3.469446951953614e-18
0.000244140625                2.9802321943606103e-8     2.9802321943606116e-8    1.3234889800848443e-23
3.0517578125e-5               4.656612873077393e-10    4.6566128719931904e-10    1.0842021724855044e-19
3.814697265625e-6             7.275957614183426e-12     7.275957614156956e-12    2.6469779601696886e-23
4.76837158203125e-7          1.1368683772161603e-13    1.1368683772160957e-13     6.462348535570529e-27
5.960464477539063e-8         1.7763568394002505e-15    1.7763568394002489e-15    1.5777218104420236e-30
7.450580596923828e-9                            0.0    2.7755575615628

Dla każdej wartości $x = 8^{-1}, 8^{-2}, 8^{-3} ... 8^{-i}$ komputer oblicza różne wartości funkcji `f()` i `g()`, pomimo tego że w matematycznym sensie funckcje są takie same. Zaczynając od $i = 9$, funkcja `f(x)` zaczyna zwracać tylko $0.0$, zatem można wznioskować że funkcja `g()` daje bardziej wiarygodne wyniki. Wynika to z tego, że funkcja `f()` wykonuje wewnętrzne działania na bliskich siebie liczbach (odejmowanie, pierwiastkowanie), co powoduje o wiele szybszą kumulację błędu. Natomaist dopóki obie funkcje zwracały wyniki, były one podobne, tj skala błędu pomiędzy nimy była kwadratowo mniejsza niż poszczególne wyniki (dla wyniku rzędu $10^i$ błąd był rzędu $10^{2i}$).

## Zad 7

*Przybliżoną wartość pochodnej f(x) w punkcie x można obliczyć za pomocą następującego wzoru $f'(x_0) \simeq \tilde{f'}(x_0) = \frac{f(x_0+h)-f(x_0)}{h}$. Skorzystać z tego wzoru do obliczenia w języku Julia w arytmetyce Float64 przybliżonej wartości pochodnej funkcji $f(x) = sin x + cos 3x$ w punkcie $x_0 = 1$ oraz błędów $|f'(x_0)-\tilde{f'}(x_0)|$ dla $h=2^{-n}$, $n \in {0, 1, 2, \dots, 54}$. Jak wytłumaczyć, że od pewnego momentu zmniejszanie wartości h nie poprawia przybliżenia wartości pochodnej? Jak zachowują się wartości 1+h? Obliczone przybliżenia pochodnej porównać z dokładną wartością pochodnej, tj. zwróć uwagę na błędy $|f'(x_0)-\tilde{f'}(x_0)|$.*

In [26]:
# Kamil Matejuk, 250135
using Printf

function f(x)
    return sin(x) + cos(3*x)
end

function dirivative(x0, h)
    @assert h != 0
    return convert(Float64, (f(x0+h) - f(x0))/h)
end

real_df = 0.11694228168853805109
println("n                            h        rounded derivative                     error")
for n in 0:54
    h = Float64(2)^(-n)
    df = dirivative(1, h)
    error = abs(real_df - df)
    println("$(@sprintf("%-4s", n)) $(@sprintf("%25s", h)) $(@sprintf("%25s", df)) $(@sprintf("%25s", error))")
end

n                            h        rounded derivative                     error
0                          1.0        2.0179892252685967        1.9010469435800585
1                          0.5        1.8704413979316472         1.753499116243109
2                         0.25        1.1077870952342974        0.9908448135457594
3                        0.125        0.6232412792975817        0.5062989976090436
4                       0.0625        0.3704000662035192       0.25345778451498113
5                      0.03125       0.24344307439754687       0.12650079270900882
6                     0.015625       0.18009756330732785       0.06315528161878979
7                    0.0078125        0.1484913953710958      0.031549113682557736
8                   0.00390625        0.1327091142805159       0.01576683259197785
9                  0.001953125        0.1248236929407085      0.007881411252170442
10                0.0009765625       0.12088247681106168      0.003940195122523624
11  

Dokładna wartość pochodnej dla podanej funkcji wynosi:<br/>
$\tilde{f'}(x) = \frac{d}{dx}(sin(x)+cos(3x)) = cos(x)-3sin(3x)$<br/>
$\tilde{f'}(1) = cos(1)-3sin(3) = 0.11694228168853805109...$<br/>
Występuje w niej też błąd, ponieważ po wprowadzeniu do systemu też została zaokraglona, ale zostanie on pominięty w rozważaniu, ponieważ skupię się na zaokrąglaniu liczby podczas liczenia pochodnej ze wzoru na pochodną w punkcie.<br/>
Najmniejszy błąd otrzymano dla wartości n=28, h=3.725290298461914e-9. Dla każdego wyższego $n$ błąd bezwzględny rósł, aż do momentu gdy obliczanie zaokrąglanej pochodnej zwracało wartość $0.0$, ponieważ liczby `h` były już na tyle małe, że w obliczaniu pochodnej w punkcie następowało odejmowanie prawie takich samych liczb `f(x)` i `f(x+h)` co powodowało kumulowanie się błędu przy redukcji cyfr znaczących.